# Run the CICI SAFE Client

#### Configure the Envrionment

In [ ]:
# Set up user's project (user's can be multiple ones, so there is no default currently)
export OS_PROJECT_NAME='CH-819993'

# Set region (again, no default currently)
export OS_REGION_NAME='CHI@TACC'

# Set chameleon keypair name and path to the private ssh key
export SSH_KEY_NAME=${USERNAME}-jupyter
export SSH_PRIVATE_KEY=${HOME}/.ssh/id_rsa

# Set the reservations to use.  
# CONTROLLER_RESERVATION_ID can be for any type of node
export CLIENT_RESERVATION_ID='c3f9c003-888b-4dbd-86c6-e79a388d136a'

#Set the working directory
export WORKING_DIR=${HOME}/work/ExoGENI-Client

#Set GENI CERT file
export GENI_CERT=${HOME}/work/geni.pem

export SDX_PUBLIC_IP='129.114.108.12'
export RIAK_PUBLIC_IP='129.114.108.234'

#### Start ExoGENI Client

In [ ]:
cd $WORKING_DIR
CLIENT_NETWORK='sharednet1'
CLIENT_NODE_NAME=${USERNAME}-client1
CLIENT_STACK_NAME=${USERNAME}-client1
GIT_BRANCH='tridentcom'
SLICE_NAME=c0-${USERNAME}
IP_PREFIX='192.168.10.1/24'
CLIENT_SITE='UFL'
SAFE_KEYPAIR='key_p10'

echo Creating SDX client. This will take several minutes!
openstack stack create  --max-width 80 \
        --template ${WORKING_DIR}/client.yaml \
        --parameter "key_name=${SSH_KEY_NAME}" \
        --parameter "reservation_id=${CLIENT_RESERVATION_ID}" \
        --parameter "network_name=${CLIENT_NETWORK}" \
        --parameter "client_name=${CLIENT_NODE_NAME}" \
        --parameter "github_branch=${GIT_BRANCH}" \
        --parameter "riak_ip=${RIAK_PUBLIC_IP}" \
        --parameter "slice_name=${SLICE_NAME}"\
        --parameter "ip_prefix=${IP_PREFIX}"\
        --parameter "client_site=${CLIENT_SITE}"\
        --parameter "safe_keypair=${SAFE_KEYPAIR}"\
        --parameter "sdx_server=${SDX_PUBLIC_IP}:8888"\
        --wait \
        ${CLIENT_STACK_NAME}

In [ ]:
echo SDX client creation complete! 
echo ${CLIENT_NODE_NAME} status `openstack server show  --format value -c status ${CLIENT_NODE_NAME}`

CLIENT_PUBLIC_IP=`openstack server show  --format value -c addresses ${CLIENT_NODE_NAME} | cut -d " " -f 2`
echo The client public IP is $CLIENT_PUBLIC_IP


#Poll for node to be ready for ssh
while [[ true ]]
do 
	ssh -q -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${CLIENT_PUBLIC_IP} \
    exit
    
    if [[ "$?" == "0" ]]; then break; fi
    
    echo Node not ready for ssh. retrying in 5 seconds.
    sleep 5
done

echo SDX node ready for ssh

In [ ]:
#Copy GENI pem file to node
REMOTE_GENI_CERT=/home/cc/geni.pem

echo Copying GENI cert to cc@$CLIENT_PUBLIC_IP:~/geni.pem
scp -q -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    ${GENI_CERT} \
    cc@${CLIENT_PUBLIC_IP}:${REMOTE_GENI_CERT}
echo Done

In [ ]:
#Tail log until complete. Note: YOU MUST MANUALLY STOP THIS TO CONTINUE!
ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${CLIENT_PUBLIC_IP} \
    sudo tail -n 20 /home/cc/boot.log

#### Create SAFE keypair

In [ ]:
SAFE_KEYPAIR="client1"

ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${CLIENT_PUBLIC_IP} \
    'sudo docker exec -itd safe /bin/bash /root/safe_keygen.sh '${SAFE_KEYPAIR}' /root/safe/safe-server/src/main/resources/prdnsmall'

res=$(curl http://${CLIENT_PUBLIC_IP}:7777/whoami \
     -H "Cntent-Type:application/json" \
     -d '{"principal":"'${SAFE_KEYPAIR}'","methodParams":[]}'
     )
     
principalId=$(echo $res | cut -d"'" -f 2)

echo principalId for ${SAFE_KEYPAIR} is $principalId


In [ ]:
# init safe sets for the new keypair, post policies
echo ${principalId}
echo ${CLINET_PUBLIC_IP}
ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${CLIENT_PUBLIC_IP} \
    "sudo ${REMOTE_BIN_DIR}/AuthorityMock init ${principalId} tag0 ${CLIENT_PUBLIC_IP}"

#### Use AuorityMock to make Delegations
##### Copy and paste those delegation tokens and add those tokens to safesets of the new keypair

In [ ]:
# Copy and pase each line from the output of authority to P
PARAMS='updateTagSet c0WrmnifdojXcYv5zS1dCSSHHKY9rbAfHbjllwZxj14= 0_ah37_Nyt8Xgqq1JFHfAD3TA9Mrx0WpLgrRx2w7Dgc=:tag0'
ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${CLIENT_PUBLIC_IP} \
    "sudo ${REMOTE_BIN_DIR}/AuthorityMock update ${principalId} ${PARAMS} ${CLIENT_PUBLIC_IP}"

In [ ]:
LOCAL_CLIENT_CONFIG=${WORKING_DIR}/client1.conf

REMOTE_CLIENT_CONFIG=/home/cc/client1.conf
REMOTE_WORKING_DIR=/home/cc
REMOTE_BIN_DIR=${REMOTE_WORKING_DIR}/CICI-SAFE/SDX-Simple/SAFE_SDX/target/appassembler/bin

cat << EOF > $LOCAL_CLIENT_CONFIG
config {
slicename="${SLICE_NAME}"
type="client"
sshkey="/root/.ssh/id_rsa"
safekey="${SAFE_KEYPAIR}"
safeserver="127.0.0.1"
safe=true
serverinslice=false
exogenipem="${REMOTE_GENI_CERT}"
exogenism="https://geni.renci.org:11443/orca/xmlrpc"
scriptsdir="${REMOTE_WORKING_DIR}/CICI-SAFE/SDX-Simple/SAFE_SDX/src/main/resources/scripts/"
resourcedir="${REMOTE_WORKING_DIR}/CICI-SAFE/SDX-Simple/SAFE_SDX/src/main/resources/"
ipprefix="${IP_PREFIX}"
serverurl="http://${SDX_PUBLIC_IP}:8888/"
routersite="${CLIENT_SITE}"
}
EOF

echo Copying SDX config to cc@$CLIENT_PUBLIC_IP
scp -q -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    $LOCAL_CLIENT_CONFIG \
    cc@${CLIENT_PUBLIC_IP}:${REMOTE_CLIENT_CONFIG}
echo Done

In [ ]:
echo Start Client Slice

echo $REMOTE_CLIENT_CONFIG

ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${CLIENT_PUBLIC_IP} \
    sudo ${REMOTE_BIN_DIR}/SafeSdxClientSliceServer -c $REMOTE_CLIENT_CONFIG

echo "Slice creation done"
#cd ${HOME}/work/CICI-SAFE/SDX-Simple
#./scripts/build.sh
#./scripts/createclientslice.sh -c $LOCAL_CLIENT_CONFIG

#### Client 1 stitch to SDX

In [ ]:
ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${CLIENT_PUBLIC_IP} \
    "sudo ${REMOTE_BIN_DIR}/SafeSdxExogeniClient -c ${REMOTE_CLIENT_CONFIG} -e 'stitch CNode1 192.168.10.2 192.168.10.1/24'"

#### Client 1 advertise its IP prefix

In [ ]:
ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${CLIENT_PUBLIC_IP} \
    "sudo ${REMOTE_BIN_DIR}/SafeSdxExogeniClient -c ${REMOTE_CLIENT_CONFIG} -e 'route 192.168.10.1/24 192.168.10.2'"


#### Connect client 1 and client 2

In [ ]:
CLIENT_CMD="\'link 192.168.10.1/24 192.168.20.1/24\'"

ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${CLIENT_PUBLIC_IP} \
    "sudo ${REMOTE_BIN_DIR}/SafeSdxExogeniClient -c ${REMOTE_CLIENT_CONFIG} -e ${CLIENT_CMD}"

#### Undo stitching

In [ ]:
CLIENT_CMD="\'unstitch CNode1\'"

ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${CLIENT_PUBLIC_IP} \
    "sudo ${REMOTE_BIN_DIR}/SafeSdxExogeniClient -c ${REMOTE_CLIENT_CONFIG} -e ${CLIENT_CMD}"

## Clean up your resources

#### Delete DataPlane Slice

In [ ]:
echo Delete Client Slice

echo $REMOTE_CLIENT_CONFIG

ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${CLIENT_PUBLIC_IP} \
    "sudo ${REMOTE_BIN_DIR}/SafeSdxClientSliceServer -c $REMOTE_CLIENT_CONFIG -d"

#### Delete Client Stack

In [ ]:
echo Deleting stack ${CLIENT_STACK_NAME}
openstack stack delete -y --wait ${CLIENT_STACK_NAME}
echo Stack ${CLIENT_STACK_NAME} deleted